In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle as pkl
import csv

In [3]:
with open("diffs.pkl",'rb') as f:
    diffs = pkl.load(f)

In [60]:
# Saving diffs file to csv
with open("test_files.csv",'w') as f:
    wr = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    # Write header
    wr.writerow( ['path to geo-referenced image 1','path to geo-referenced image 2','image difference'] )
    for row in diffs:
        wr.writerow( row )
        

In [57]:
loaded_data = []
with open("test_files2.csv",'r') as f:
    lines = f.readlines()
    try:
        row = next( csv.reader(lines[:1]) )
        loaded_data.append( (row[0],row[1],float(row[2])) )
    except ValueError as ex:
        pass
    
    for row in csv.reader(lines[1:]):
        loaded_data.append( (row[0],row[1],float(row[2])) )

In [70]:
raise NotImplementedError()

NotImplementedError: Please, fuck off

In [71]:
[1,2,][3]

IndexError: list index out of range

In [1]:
import testgen
import importlib
importlib.reload(testgen)

<module 'testgen' from '/home/artef/IdeaProjects/prrnmosaic/python_scripts/testgen.py'>

In [2]:
samplegen = testgen.TestSampleGen_from_csv('test_files.csv')

In [3]:
for i1, i2 in samplegen.most_alike():
    break

OSError: File "/media/artef/96FEEE12FEEDEB07/Users/artef/Maps/tiles/LC08_L1TP_178021_20160926_20170319_01_T1_B_MERGED/LC08_L1TP_178021_20160926_20170319_01_T1_B_MERGED_TILE_-1214_-1226.TIF" could not be read